In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_columns', None)   
pd.set_option('display.width', None)         

artifacts = pd.read_csv("artifacts.csv")
artifact_release = pd.read_csv("artifact_release.csv")
release = pd.read_csv("releases.csv")


In [ ]:
artifact_release['artifact_id'].nunique()

In [ ]:
release['release_timestamp'] = pd.to_datetime(release['release_timestamp'], unit='ms')


In [ ]:
# Merge artifacts with their releases
artifact_releases = pd.merge(artifact_release, release, on='release_id', how='inner')
artifact_releases = pd.merge(artifact_releases, artifacts[['artifact_id']], on='artifact_id', how='inner')

In [ ]:
# Define the date range
start_date = '2014-09-04'
end_date = '2024-09-04'

# Filter the release history to include only records within 2014-2024
filtered_release_history = artifact_releases[
    (artifact_releases['release_timestamp'] >= start_date) &
    (artifact_releases['release_timestamp'] <= end_date)
]
filtered_release_history = filtered_release_history.sort_values(by=['artifact_id', 'release_timestamp'])
filtered_release_history

In [ ]:
filtered_release_history['artifact_id'].nunique()

In [ ]:

# Group by artifact_id and get the first and last release date per project
release_date_range = filtered_release_history.groupby('artifact_id')['release_timestamp'].agg(['min', 'max'])
# Group by artifact_id and get the first and last release date per project
release_date_range = filtered_release_history.groupby('artifact_id')['release_timestamp'].agg(['min', 'max'])

# Filter to include only projects with both first and last release dates within 2014-2024
valid_projects = release_date_range[
    (release_date_range['min'] >= start_date) &
    (release_date_range['max'] <= end_date)
].index

# Filter the original dataframe to include only valid projects
filtered_release_history = filtered_release_history[filtered_release_history['artifact_id'].isin(valid_projects)]
filtered_release_history

In [ ]:
filtered_release_history['artifact_id'].nunique()

In [ ]:
new_project_cutoff = pd.to_datetime('2023-08-30')


# Group by artifact_id to get the first and last release dates and count of releases per project
release_date_range = filtered_release_history.groupby('artifact_id')['release_timestamp'].agg(['min', 'max', 'count'])


In [ ]:
release_date_range

In [ ]:
# Apply filters:
# 1. Include projects with more than 1 release
# 2. Include projects with the first release date before August 30, 2023
valid_projects = release_date_range[
    (release_date_range['count'] > 1) &  # More than 1 release
    (release_date_range['min'] < new_project_cutoff)  # First release before August 30, 2023
].index

valid_projects

In [ ]:
filtered_release_history = filtered_release_history[filtered_release_history['artifact_id'].isin(valid_projects)]
filtered_release_history

In [ ]:
filtered_release_history['artifact_id'].nunique()

In [ ]:
#filtered_release_history.to_csv('full_release_history.csv', index=False)

In [ ]:

# Extract year from release_timestamp
filtered_release_history['release_year'] = filtered_release_history['release_timestamp'].dt.year

# Define the abandonment cutoff date
abandonment_cutoff = pd.to_datetime('2022-09-04')

# Identify the last release date per project
last_release_per_project = filtered_release_history.groupby('artifact_id')['release_timestamp'].max()

# Identify abandoned projects (last release on or before the abandonment cutoff date)
abandoned_projects = last_release_per_project[last_release_per_project <= abandonment_cutoff].index

# Find the first release year for each project (artifact_id)
first_release_year = (
    filtered_release_history.groupby('artifact_id')['release_year']
    .min()
    .reset_index()
    .rename(columns={'release_year': 'creation_year'})
)

# Filter for abandoned projects in the original dataframe to get their release history
abandoned_release_history = filtered_release_history[filtered_release_history['artifact_id'].isin(abandoned_projects)]

# Get the last release year for each abandoned project
abandoned_projects_last_year = (
    abandoned_release_history.groupby('artifact_id')['release_timestamp']
    .max()
    .dt.year
    .reset_index()
    .rename(columns={'release_timestamp': 'abandonment_year'})
)

# Initialize sets and list for cumulative tracking
cumulative_projects = set()
yearly_data = []

# Iterate over years and calculate yearly data
for year in sorted(filtered_release_history['release_year'].unique()):
    # Projects created in the current year
    new_projects = set(first_release_year[first_release_year['creation_year'] == year]['artifact_id'])
    
    # Continuing projects: Exclude projects abandoned in the previous year
    if year > min(filtered_release_history['release_year'].unique()):  # Avoid looking for abandoned projects before the first year
        abandoned_last_year = set(
            abandoned_projects_last_year[abandoned_projects_last_year['abandonment_year'] == (year - 1)]['artifact_id']
        )
        cumulative_projects -= abandoned_last_year  # Remove abandoned projects from the cumulative set
    
    # Continuing projects: Projects in cumulative set not newly created this year
    continuing_projects = cumulative_projects - new_projects
    
    # Add current year's new projects to the cumulative set
    cumulative_projects.update(new_projects)
    
    # Total projects for the year (new + continuing)
    total_projects = len(new_projects) + len(continuing_projects)
    
    # Update cumulative set for tracking unique projects
    cumulative_unique_projects = len(cumulative_projects)
    
    # Append yearly data
    yearly_data.append({
        'year': year,
        'continuing_projects': len(continuing_projects),
        'new_projects': len(new_projects),
        'total_projects_active': total_projects,
        #'cumulative_unique_projects': cumulative_unique_projects
    })

# Convert to DataFrame
yearly_projects_df = pd.DataFrame(yearly_data)

# Incorporate abandonment data
abandoned_yearly_data = []
cumulative_abandoned_projects = set()

for year in sorted(filtered_release_history['release_year'].unique()):
    # Abandoned projects in the current year
    abandoned_in_year = set(
        abandoned_projects_last_year[abandoned_projects_last_year['abandonment_year'] == year]['artifact_id']
    )
    # Update cumulative abandoned projects
    cumulative_abandoned_projects.update(abandoned_in_year)
    
    # Append abandonment data
    abandoned_yearly_data.append({
        'year': year,
        'total_unique_abandoned': len(abandoned_in_year),
        'cumsum_total_unique_abandoned': len(cumulative_abandoned_projects)
    })

# Merge yearly abandonment data with the project data
abandoned_yearly_df = pd.DataFrame(abandoned_yearly_data)
yearly_projects_df = yearly_projects_df.merge(abandoned_yearly_df, on='year', how='left')

# Fill NaNs with 0s for years without abandonment
yearly_projects_df.fillna(0, inplace=True)

# Convert counts to integers for clarity
yearly_projects_df = yearly_projects_df.astype({
    'continuing_projects': 'int',
    'new_projects': 'int',
    'total_projects_active': 'int',
   # 'cumulative_unique_projects': 'int',
    'total_unique_abandoned': 'int',
    'cumsum_total_unique_abandoned': 'int'
})

# Display the updated DataFrame
yearly_projects_df



In [ ]:
#yearly_projects_df.to_latex('yearly_projects_df',index=False)

In [ ]:
# Calculate the abandonment rate as total_unique_abandoned / total_projects
yearly_projects_df['abandonment_rate'] = yearly_projects_df['total_unique_abandoned'] / yearly_projects_df['total_projects_active']

# Display the updated DataFrame
yearly_projects_df


In [ ]:
import matplotlib.pyplot as plt

# Calculate cumulative total projects
yearly_projects_df['cumulative_total_projects'] = yearly_projects_df['total_projects_active'] + yearly_projects_df['cumsum_total_unique_abandoned']

# Plot cumulative total projects vs abandoned projects
plt.figure(figsize=(10, 6))
plt.plot(yearly_projects_df['year'], yearly_projects_df['cumulative_total_projects'], label='Cumulative total libraries', marker='o')
plt.plot(yearly_projects_df['year'], yearly_projects_df['cumsum_total_unique_abandoned'], label='Cumulative abandoned libraries', marker='o')

# Add labels, title, legend, and grid
plt.xlabel('Year')
plt.ylabel('Number of projects')
plt.title('Trend of Total (active+abandoned) and abandoned libraries over time')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
# Classify projects as abandoned or active
# abandoned_projects = last_release_per_project[last_release_per_project <= abandonment_cutoff].index
active_projects = last_release_per_project[last_release_per_project > abandonment_cutoff].index
active_release_history = filtered_release_history[filtered_release_history['artifact_id'].isin(active_projects)]
abandoned_release_history

In [ ]:
abandoned_release_history['artifact_id'].nunique()

In [ ]:
active_release_history['artifact_id'].nunique()

In [ ]:
# Identify the last release date per project
last_release_per_project = filtered_release_history.groupby('artifact_id')['release_timestamp'].max()
last_release_per_project

In [ ]:
# Sort by artifact_id and release_timestamp to calculate intervals in both active and abandoned projects
active_release_history = active_release_history.sort_values(by=['artifact_id', 'release_timestamp'])
abandoned_release_history = abandoned_release_history.sort_values(by=['artifact_id', 'release_timestamp'])


In [ ]:
active_release_history

In [ ]:
import pandas as pd

# Ensure release_timestamp is in datetime format
abandoned_release_history['release_timestamp'] = pd.to_datetime(abandoned_release_history['release_timestamp'])

# Step 1: Determine first and last release dates for each project (artifact)
project_lifespans = (
    abandoned_release_history
    .sort_values('release_timestamp')
    .groupby('artifact_id')
    .agg(
        first_release_date=('release_timestamp', 'first'),
        last_release_date=('release_timestamp', 'last')
    )
    .reset_index()
)

# Calculate lifespan in days
project_lifespans['lifespan_days'] = (project_lifespans['last_release_date'] - project_lifespans['first_release_date']).dt.days

# Step 2: Segment by cohort (initial release year)
project_lifespans['cohort_year'] = project_lifespans['first_release_date'].dt.year

# Step 3: Calculate mean and median lifespan by cohort
cohort_lifespans = (
    project_lifespans
    .groupby('cohort_year')
    .agg(
        mean_lifespan_days=('lifespan_days', 'mean'),
        median_lifespan_days=('lifespan_days', 'median'),
        num_projects=('artifact_id', 'nunique')
    )
    .reset_index()
)

# Display the results
cohort_lifespans

In [ ]:
import pandas as pd
import numpy as np


# Define the abandonment cutoff date
abandonment_cutoff = pd.to_datetime('2022-09-04')

# Determine if a project is abandoned
project_lifespans['abandoned'] = project_lifespans['last_release_date'] <= abandonment_cutoff

# Calculate the abandonment year
project_lifespans['abandonment_year'] = np.where(
    project_lifespans['abandoned'],
    project_lifespans['last_release_date'].dt.year,
    np.nan
)

# Calculate the age at abandonment (years since first release)
project_lifespans['years_to_abandonment'] = (
    project_lifespans['abandonment_year'] - project_lifespans['cohort_year']
)

# Filter only the abandoned projects
abandoned_projects = project_lifespans[project_lifespans['abandoned']].copy()

# Summarize abandonment counts by cohort year and years to abandonment
abandonment_summary = (
    abandoned_projects
    .groupby(['cohort_year', 'years_to_abandonment'])
    .agg({'artifact_id': 'nunique'})
    .reset_index()
    .rename(columns={'artifact_id': 'num_abandoned_projects'})
)

# Display the abandonment summary
abandonment_summary


In [ ]:
import pandas as pd
import numpy as np


# Define the abandonment cutoff date
abandonment_cutoff = pd.to_datetime('2022-09-04')

# Determine if a project is abandoned
project_lifespans['abandoned'] = project_lifespans['last_release_date'] <= abandonment_cutoff

# Calculate the abandonment year
project_lifespans['abandonment_year'] = np.where(
    project_lifespans['abandoned'],
    project_lifespans['last_release_date'].dt.year,
    np.nan
)

# Calculate the age at abandonment (years since first release)
project_lifespans['years_to_abandonment'] = (
    project_lifespans['abandonment_year'] - project_lifespans['cohort_year']
)

# Convert 'years_to_abandonment' to integer (and handle NaN)
project_lifespans['years_to_abandonment'] = project_lifespans['years_to_abandonment'].fillna(-1).astype(int)

# Filter only the abandoned projects
abandoned_projects = project_lifespans[project_lifespans['abandoned']].copy()

# Summarize abandonment counts by cohort year and years to abandonment
abandonment_summary = (
    abandoned_projects
    .groupby(['cohort_year', 'years_to_abandonment'])
    .agg(num_abandoned_projects=('artifact_id', 'nunique'))
    .reset_index()
)

# Pivot the 'abandonment_summary' DataFrame to have 'years_to_abandonment' as columns
abandonment_pivot = abandonment_summary.pivot(
    index='cohort_year',
    columns='years_to_abandonment',
    values='num_abandoned_projects'
).fillna(0)

# Optional: Select the specific years you are interested in (e.g., 1, 2, 3, 5)
milestones = [1, 2, 3]

# Ensure all milestones are included in the columns (add missing ones with zero counts)
for milestone in milestones:
    if milestone not in abandonment_pivot.columns:
        abandonment_pivot[milestone] = 0

# Sort the columns for consistency
abandonment_pivot = abandonment_pivot.sort_index(axis=1)

# Rename the columns to reflect 'abandoned_after_X_years'
abandonment_pivot.rename(columns={year: f'abandoned_after_{year}_years' for year in abandonment_pivot.columns}, inplace=True)

# Reset index to turn 'cohort_year' back into a column
abandonment_pivot.reset_index(inplace=True)

# Display the abandonment pivot table
abandonment_pivot


In [ ]:
# Calculate total number of projects per cohort
total_projects = (
    project_lifespans.groupby('cohort_year')['artifact_id']
    .nunique()
    .reset_index()
    .rename(columns={'artifact_id': 'num_projects'})
)

# Merge total projects with the abandonment pivot table
abandonment_pivot = abandonment_pivot.merge(total_projects, on='cohort_year', how='left')

# Reorder columns to place 'num_projects' after 'cohort_year' for clarity
cols = ['cohort_year', 'num_projects'] + [col for col in abandonment_pivot.columns if col not in ['cohort_year', 'num_projects']]
abandonment_pivot = abandonment_pivot[cols]

# Display the updated DataFrame
abandonment_pivot


upto this, we are only looking total number of abandoned projects per year. now to fair comparison and calculate the rates, adding the active projects below.

In [ ]:

import pandas as pd
import numpy as np

# Assume 'abandoned_release_history' and 'active_release_history' DataFrames exist

# Ensure release_timestamp is in datetime format for both datasets
abandoned_release_history['release_timestamp'] = pd.to_datetime(abandoned_release_history['release_timestamp'])
active_release_history['release_timestamp'] = pd.to_datetime(active_release_history['release_timestamp'])

# Step 1: Determine first and last release dates for each project (artifact) in both datasets
abandoned_project_lifespans = (
    abandoned_release_history
    .sort_values('release_timestamp')
    .groupby('artifact_id')
    .agg(
        first_release_date=('release_timestamp', 'first'),
        last_release_date=('release_timestamp', 'last')
    )
    .reset_index()
)

active_project_lifespans = (
    active_release_history
    .sort_values('release_timestamp')
    .groupby('artifact_id')
    .agg(
        first_release_date=('release_timestamp', 'first'),
        last_release_date=('release_timestamp', 'last')
    )
    .reset_index()
)

# Step 2: Combine both datasets into 'project_lifespans'
project_lifespans = pd.concat([abandoned_project_lifespans, active_project_lifespans], ignore_index=True)

# Step 3: Calculate lifespan in days
project_lifespans['lifespan_days'] = (
    project_lifespans['last_release_date'] - project_lifespans['first_release_date']
).dt.days

# Step 4: Segment by cohort (initial release year)
project_lifespans['cohort_year'] = project_lifespans['first_release_date'].dt.year

# Define the abandonment cutoff date
abandonment_cutoff = pd.to_datetime('2022-09-04')

# Step 5: Determine if a project is abandoned
project_lifespans['abandoned'] = project_lifespans['last_release_date'] <= abandonment_cutoff

# Step 6: Calculate the abandonment year
project_lifespans['abandonment_year'] = np.where(
    project_lifespans['abandoned'],
    project_lifespans['last_release_date'].dt.year,
    np.nan
)

# Step 7: Calculate the age at abandonment or censoring (years since first release)
project_lifespans['years_to_event'] = np.where(
    project_lifespans['abandoned'],
    project_lifespans['abandonment_year'] - project_lifespans['cohort_year'],
    abandonment_cutoff.year - project_lifespans['cohort_year']
)
project_lifespans['years_to_event'] = project_lifespans['years_to_event'].astype(int)

# Step 8: Create milestone columns to indicate abandonment by specific years
milestones = [0, 1, 2, 3]

for milestone in milestones:
    project_lifespans[f'abandoned_by_{milestone}_years'] = np.where(
        (project_lifespans['abandoned']) & (project_lifespans['years_to_event'] <= milestone),
        1,
        0
    )

# Step 9: Summarize abandonment counts by cohort year
abandonment_summary = (
    project_lifespans
    .groupby('cohort_year')
    .agg(
        total_projects=('artifact_id', 'nunique'),
        **{f'abandoned_by_{milestone}_years': (f'abandoned_by_{milestone}_years', 'sum') for milestone in milestones}
    )
    .reset_index()
)

# Step 10: Calculate active projects remaining after each milestone
for milestone in milestones:
    abandonment_summary[f'active_after_{milestone}_years'] = (
        abandonment_summary['total_projects'] - abandonment_summary[f'abandoned_by_{milestone}_years']
    )

# Optional: Reorder columns for clarity
cols = ['cohort_year', 'total_projects'] + \
       sum([[f'abandoned_by_{milestone}_years', f'active_after_{milestone}_years'] for milestone in milestones], [])
abandonment_summary = abandonment_summary[cols]

# Step 11: Display the final DataFrame
abandonment_summary



In [ ]:
# Calculate total_abandoned_projects per cohort_year
total_abandoned_projects = (
    project_lifespans[project_lifespans['abandoned']]
    .groupby('cohort_year')['artifact_id']
    .nunique()
    .reset_index()
    .rename(columns={'artifact_id': 'total_abandoned_projects'})
)

# Merge total_abandoned_projects into abandonment_summary
abandonment_summary = abandonment_summary.merge(total_abandoned_projects, on='cohort_year', how='left')

# Fill NaN with zeros (in case some cohorts have no abandoned projects)
abandonment_summary['total_abandoned_projects'] = abandonment_summary['total_abandoned_projects'].fillna(0).astype(int)

# Calculate total_active_projects
abandonment_summary['total_active_projects'] = (
    abandonment_summary['total_projects'] - abandonment_summary['total_abandoned_projects']
)

# Reorder columns for clarity
cols = ['cohort_year', 'total_projects', 'total_abandoned_projects', 'total_active_projects'] + \
       [col for col in abandonment_summary.columns if col not in ['cohort_year', 'total_projects', 'total_abandoned_projects', 'total_active_projects']]
abandonment_summary = abandonment_summary[cols]

# Display the final DataFrame
abandonment_summary


In [ ]:
abandonment_summary['total_abandoned_projects'].sum()

In [ ]:
abandonment_summary['total_active_projects'].sum()

In [ ]:
# Calculate abandonment rates for each milestone year
for milestone in milestones:
    abandonment_summary[f'abandoned_rate_{milestone}_years'] = (
        abandonment_summary[f'abandoned_by_{milestone}_years'] / abandonment_summary['total_projects']
    )

for milestone in milestones:
    abandonment_summary[f'abandoned_rate_{milestone}_years'] *= 100

# Reorder columns for clarity
rate_columns = [f'abandoned_rate_{milestone}_years' for milestone in milestones]
cols = ['cohort_year', 'total_projects', 'total_abandoned_projects', 'total_active_projects'] + \
       sum([[f'abandoned_by_{milestone}_years', f'abandoned_rate_{milestone}_years', f'active_after_{milestone}_years'] for milestone in milestones], [])
abandonment_summary = abandonment_summary[cols]

# Display the updated DataFrame
abandonment_summary


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# Define the milestones you're interested in
milestones = [0, 1, 2, 3]  # Adjust this list if you have different milestones

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot the abandonment rates for each milestone
for milestone in milestones:
    plt.plot(
        abandonment_summary['cohort_year'],
        abandonment_summary[f'abandoned_rate_{milestone}_years'],
        marker='o',
        label=f'Abandoned after {milestone} Year(s)'
    )

# Set plot title and labels
plt.title('Abandonment Rate Trend Over Cohort Years')
plt.xlabel('Cohort Year')
plt.ylabel('Abandonment Rate')

# Format the y-axis as percentages
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Add grid for better readability
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Add legend
plt.legend(title='Milestone Years')

# Show the plot
plt.show()
